In [42]:
# Code Reference https://github.com/hzxie/Pix2Vox/

In [225]:
import torch
import torchvision.models


class Encoder(torch.nn.Module):
    def __init__(self, cfg, pretrained=True):
        super(Encoder, self).__init__()
        self.cfg = cfg

        # Layer Definition
        resnet50 = torchvision.models.resnet50(pretrained=pretrained)
        self.resnet = torch.nn.Sequential(*list(resnet50.children()))[:-3]
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1024, 512, kernel_size=3),
            torch.nn.BatchNorm2d(512),
            torch.nn.ELU(),
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 512, kernel_size=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ELU(),
            torch.nn.MaxPool2d(kernel_size=3)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 256, kernel_size=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ELU()
        )

        # Don't update params in ResNet
        for param in resnet50.parameters():
            param.requires_grad = False

    def forward(self, rendering_images):
        # print(rendering_images.size())  # torch.Size([batch_size, n_views, img_c, img_h, img_w])
        rendering_images = rendering_images.permute(1, 0, 2, 3, 4).contiguous()
        rendering_images = torch.split(rendering_images, 1, dim=0)
        image_features = []

        for img in rendering_images:
            features = self.resnet(img.squeeze(dim=0))
            # print(features.size())    # torch.Size([batch_size, 512, 26, 26])
            features = self.layer1(features)
            # print(features.size())    # torch.Size([batch_size, 512, 24, 24])
            features = self.layer2(features)
            # print(features.size())    # torch.Size([batch_size, 512, 8, 8])
            features = self.layer3(features)
            # print(features.size())    # torch.Size([batch_size, 256, 8, 8])
            image_features.append(features)

        image_features = torch.stack(image_features).permute(1, 0, 2, 3, 4).contiguous()
        # print(image_features.size())  # torch.Size([batch_size, n_views, 256, 8, 8])
        return image_features

In [40]:
from torchsummary import summary

In [61]:
model = Encoder(cfg=None).cuda()
# input size = [batch_size, n_views, img_c, img_h, img_w]
encoder_outputs =model(torch.Tensor(1, 6, 3, 416, 416).cuda())

In [45]:
class Decoder(torch.nn.Module):
    def __init__(self, cfg):
        super(Decoder, self).__init__()
        self.cfg = cfg

        # Layer Definition
        self.layer1 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(2048, 512, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm3d(512),
            torch.nn.ReLU()
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(512, 128, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm3d(128),
            torch.nn.ReLU()
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(128, 32, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm3d(32),
            torch.nn.ReLU()
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(32, 8, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm3d(8),
            torch.nn.ReLU()
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(8, 1, kernel_size=1, bias=True),
            torch.nn.Sigmoid()
        )

    def forward(self, image_features):
        image_features = image_features.permute(1, 0, 2, 3, 4).contiguous()
        image_features = torch.split(image_features, 1, dim=0)
        gen_volumes = []
        raw_features = []

        for features in image_features:
            gen_volume = features.view(-1, 2048, 2, 2, 2)
            # print(gen_volume.size())   # torch.Size([batch_size, 2048, 2, 2, 2])
            gen_volume = self.layer1(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 512, 4, 4, 4])
            gen_volume = self.layer2(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 128, 8, 8, 8])
            gen_volume = self.layer3(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 32, 16, 16, 16])
            gen_volume = self.layer4(gen_volume)
            raw_feature = gen_volume
            # print(gen_volume.size())   # torch.Size([batch_size, 8, 32, 32, 32])
            gen_volume = self.layer5(gen_volume)
            # print(gen_volume.size())   # torch.Size([batch_size, 1, 32, 32, 32])
            raw_feature = torch.cat((raw_feature, gen_volume), dim=1)
            # print(raw_feature.size())  # torch.Size([batch_size, 9, 32, 32, 32])

            gen_volumes.append(torch.squeeze(gen_volume, dim=1))
            raw_features.append(raw_feature)

        gen_volumes = torch.stack(gen_volumes).permute(1, 0, 2, 3, 4).contiguous()
        raw_features = torch.stack(raw_features).permute(1, 0, 2, 3, 4, 5).contiguous()
        # print(gen_volumes.size())      # torch.Size([batch_size, n_views, 32, 32, 32])
        # print(raw_features.size())     # torch.Size([batch_size, n_views, 9, 32, 32, 32])
        return raw_features, gen_volumes


In [62]:
model = Decoder(cfg=None).cuda()
# input size = output of encoder 
raw_features, gen_volumes = model(torch.Tensor(encoder_outputs.shape).cuda())

In [51]:
raw_features.shape

torch.Size([1, 6, 9, 32, 32, 32])

In [52]:
gen_volumes.shape

torch.Size([1, 6, 32, 32, 32])

In [55]:
class Merger(torch.nn.Module):
    def __init__(self, cfg):
        super(Merger, self).__init__()
        self.cfg = cfg

        # Layer Definition
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv3d(9, 16, kernel_size=3, padding=1),
            torch.nn.BatchNorm3d(16),
            torch.nn.LeakyReLU(.2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv3d(16, 8, kernel_size=3, padding=1),
            torch.nn.BatchNorm3d(8),
            torch.nn.LeakyReLU(.2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv3d(8, 4, kernel_size=3, padding=1),
            torch.nn.BatchNorm3d(4),
            torch.nn.LeakyReLU(.2)
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv3d(4, 2, kernel_size=3, padding=1),
            torch.nn.BatchNorm3d(2),
            torch.nn.LeakyReLU(.2)
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.Conv3d(2, 1, kernel_size=3, padding=1),
            torch.nn.BatchNorm3d(1),
            torch.nn.LeakyReLU(.2)
        )

    def forward(self, raw_features, coarse_volumes):
        n_views_rendering = coarse_volumes.size(1)
        raw_features = torch.split(raw_features, 1, dim=1)
        volume_weights = []

        for i in range(n_views_rendering):
            raw_feature = torch.squeeze(raw_features[i], dim=1)
            # print(raw_feature.size())       # torch.Size([batch_size, 9, 32, 32, 32])

            volume_weight = self.layer1(raw_feature)
            # print(volume_weight.size())     # torch.Size([batch_size, 16, 32, 32, 32])
            volume_weight = self.layer2(volume_weight)
            # print(volume_weight.size())     # torch.Size([batch_size, 8, 32, 32, 32])
            volume_weight = self.layer3(volume_weight)
            # print(volume_weight.size())     # torch.Size([batch_size, 4, 32, 32, 32])
            volume_weight = self.layer4(volume_weight)
            # print(volume_weight.size())     # torch.Size([batch_size, 2, 32, 32, 32])
            volume_weight = self.layer5(volume_weight)
            # print(volume_weight.size())     # torch.Size([batch_size, 1, 32, 32, 32])

            volume_weight = torch.squeeze(volume_weight, dim=1)
            # print(volume_weight.size())     # torch.Size([batch_size, 32, 32, 32])
            volume_weights.append(volume_weight)

        volume_weights = torch.stack(volume_weights).permute(1, 0, 2, 3, 4).contiguous()
        volume_weights = torch.softmax(volume_weights, dim=1)
        # print(volume_weights.size())        # torch.Size([batch_size, n_views, 32, 32, 32])
        # print(coarse_volumes.size())        # torch.Size([batch_size, n_views, 32, 32, 32])
        coarse_volumes = coarse_volumes * volume_weights
        coarse_volumes = torch.sum(coarse_volumes, dim=1)

        return torch.clamp(coarse_volumes, min=0, max=1)

In [67]:
model = Merger(cfg=None).cuda()
merger_volumns = model(torch.Tensor(raw_features.shape).cuda(), torch.Tensor(gen_volumes.shape).cuda())

In [64]:
class Refiner(torch.nn.Module):
    def __init__(self, cfg):
        super(Refiner, self).__init__()
        self.cfg = cfg

        # Layer Definition
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv3d(1, 32, kernel_size=4, padding=2),
            torch.nn.BatchNorm3d(32),
            torch.nn.LeakyReLU(0.2),
            torch.nn.MaxPool3d(kernel_size=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv3d(32, 64, kernel_size=4, padding=2),
            torch.nn.BatchNorm3d(64),
            torch.nn.LeakyReLU(0.2),
            torch.nn.MaxPool3d(kernel_size=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv3d(64, 128, kernel_size=4, padding=2),
            torch.nn.BatchNorm3d(128),
            torch.nn.LeakyReLU(0.2),
            torch.nn.MaxPool3d(kernel_size=2)
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(8192, 2048),
            torch.nn.ReLU()
        )
        self.layer5 = torch.nn.Sequential(
            torch.nn.Linear(2048, 8192),
            torch.nn.ReLU()
        )
        self.layer6 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(128, 64, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm3d(64),
            torch.nn.ReLU()
        )
        self.layer7 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(64, 32, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm3d(32),
            torch.nn.ReLU()
        )
        self.layer8 = torch.nn.Sequential(
            torch.nn.ConvTranspose3d(32, 1, kernel_size=4, stride=2, bias=True, padding=1),
            torch.nn.Sigmoid()
        )

    def forward(self, coarse_volumes):
        volumes_32_l = coarse_volumes.view((-1, 1, 32, 32, 32))
        # print(volumes_32_l.size())       # torch.Size([batch_size, 1, 32, 32, 32])
        volumes_16_l = self.layer1(volumes_32_l)
        # print(volumes_16_l.size())       # torch.Size([batch_size, 32, 16, 16, 16])
        volumes_8_l = self.layer2(volumes_16_l)
        # print(volumes_8_l.size())        # torch.Size([batch_size, 64, 8, 8, 8])
        volumes_4_l = self.layer3(volumes_8_l)
        # print(volumes_4_l.size())        # torch.Size([batch_size, 128, 4, 4, 4])
        flatten_features = self.layer4(volumes_4_l.view(-1, 8192))
        # print(flatten_features.size())   # torch.Size([batch_size, 2048])
        flatten_features = self.layer5(flatten_features)
        # print(flatten_features.size())   # torch.Size([batch_size, 8192])
        volumes_4_r = volumes_4_l + flatten_features.view(-1, 128, 4, 4, 4)
        # print(volumes_4_r.size())        # torch.Size([batch_size, 128, 4, 4, 4])
        volumes_8_r = volumes_8_l + self.layer6(volumes_4_r)
        # print(volumes_8_r.size())        # torch.Size([batch_size, 64, 8, 8, 8])
        volumes_16_r = volumes_16_l + self.layer7(volumes_8_r)
        # print(volumes_16_r.size())       # torch.Size([batch_size, 32, 16, 16, 16])
        volumes_32_r = (volumes_32_l + self.layer8(volumes_16_r)) * 0.5
        # print(volumes_32_r.size())       # torch.Size([batch_size, 1, 32, 32, 32])

        return volumes_32_r.view((-1, 32, 32, 32))

In [71]:
model = Refiner(cfg=None).cuda()
refiner_outputs = model(torch.Tensor(merger_volumns.shape).cuda())
refiner_outputs.shape

torch.Size([1, 32, 32, 32])

In [213]:
class Mapper(torch.nn.Module):
    def __init__(self, cfg):
        super(Mapper, self).__init__()
        self.cfg = cfg
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 8, kernel_size=9, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(0.2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(8, 8, kernel_size=4, stride=4, bias=True, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(),
            
            torch.nn.ConvTranspose2d(8, 8, kernel_size=2, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm2d(8),
            torch.nn.ReLU(),
            
            torch.nn.ConvTranspose2d(8, 4, kernel_size=2, stride=2, bias=True, padding=1),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(),
            
            torch.nn.ConvTranspose2d(4, 4, kernel_size=2, stride=2, bias=True, padding=2),
            torch.nn.BatchNorm2d(4),
            torch.nn.ReLU(),
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 1, kernel_size=1, padding=0),
            torch.nn.BatchNorm2d(1),
            torch.nn.ReLU(0.2)
        )
        
        
        
    def forward(self, volumes):
        batch_size = volumes.shape[0]
        x = self.layer1(volumes)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [214]:
model = Mapper(cfg=None).cuda()
model(torch.Tensor(refiner_outputs.shape).cuda()).shape

torch.Size([1, 1, 800, 800])

### implementation

In [216]:
encoder = Encoder(cfg=None).cuda()
decoder = Decoder(cfg=None).cuda()
merger = Merger(cfg=None).cuda()
refiner  = Refiner(cfg=None).cuda()
mapper = Mapper(cfg=None).cuda()


In [218]:
# input size = [batch_size, n_views, img_c, img_h, img_w]
encoder_outputs=encoder(torch.Tensor(1, 6, 3, 416, 416).cuda())
raw_features, gen_volumes = decoder(encoder_outputs)
merger_volumns = merger(raw_features, gen_volumes)
refiner_columns = refiner(merger_volumns)
outputs = mapper(refiner_columns)

In [219]:
outputs.shape

torch.Size([1, 1, 800, 800])

### See implementation details:

https://github.com/hzxie/Pix2Vox/blob/f31ade6142f07e1d21c9c1c02b23e41410a9b230/core/train.py#L27

In [222]:
class pix2vox(torch.nn.Module):
    def __init__(self, cfg, pretrained=True):
        super(pix2vox, self).__init__()
        self.cfg = cfg
        self.encoder = Encoder(self.cfg, pretrained=pretrained)
        self.decoder = Decoder(self.cfg)
        self.merger = Merger(self.cfg)
        self.refiner = Refiner(self.cfg)
        self.mapper = Mapper(self.cfg)
        
        self.init_weights()

    def forward(self, inputs):
        encoder_outputs = encoder(inputs)
        raw_features, gen_volumes = decoder(encoder_outputs)
        merger_volumns = merger(raw_features, gen_volumes)
        refiner_columns = refiner(merger_volumns)
        outputs = mapper(refiner_columns)
        return outputs
    
    
    def init_weights(m):
        if type(m) == torch.nn.Conv2d or type(m) == torch.nn.Conv3d or type(m) == torch.nn.ConvTranspose3d:
            torch.nn.init.kaiming_normal_(m.weight)
            if m.bias is not None:
                torch.nn.init.constant_(m.bias, 0)
        elif type(m) == torch.nn.BatchNorm2d or type(m) == torch.nn.BatchNorm3d:
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0)
        elif type(m) == torch.nn.Linear:
            torch.nn.init.normal_(m.weight, 0, 0.01)
            torch.nn.init.constant_(m.bias, 0)

In [227]:
mdl = pix2vox(None, pretrained=True).cuda()

In [229]:
mdl(torch.Tensor(1, 6, 3, 416, 416).cuda())

tensor([[[[0.0000, 1.4334, 1.7516,  ..., 0.5580, 0.0000, 1.1863],
          [0.0000, 0.0000, 1.2064,  ..., 0.0000, 0.0000, 0.0000],
          [1.7576, 2.0799, 0.0000,  ..., 3.8159, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.5333,  ..., 0.0000, 0.0000, 0.0000],
          [0.6271, 0.6815, 0.0714,  ..., 3.9484, 1.0071, 0.0000],
          [0.2754, 0.0000, 0.0833,  ..., 0.0000, 0.5345, 0.0000]]]],
       device='cuda:0', grad_fn=<ReluBackward1>)